In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
import matplotlib
matplotlib.use('Agg') # No pictures displayed 
import pylab
import librosa
import librosa.display
import numpy as np


df = pd.read_csv('train.tsv', sep="\t" , header=0)

In [38]:
df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accent
102,b7d65328f769972d7c622e2bc5adb5f882b15313f8c848...,common_voice_en_544828.mp3,Why does my bee always come last in the race?,3,0,teens,male,us
103,b7d65328f769972d7c622e2bc5adb5f882b15313f8c848...,common_voice_en_544830.mp3,I need the weather close-by in Mississippi,2,0,teens,male,us
108,c6088565cca9ad7c8c109796206b491f0aeb672c57dfa6...,common_voice_en_121231.mp3,A woman is wearing a bright orange and brown s...,3,0,teens,male,us
109,c6088565cca9ad7c8c109796206b491f0aeb672c57dfa6...,common_voice_en_121232.mp3,Three men sitting in a room looking at their o...,2,0,teens,male,us
110,c6088565cca9ad7c8c109796206b491f0aeb672c57dfa6...,common_voice_en_121233.mp3,A dog is on a leash by a tree,2,0,teens,male,us


In [2]:
df = df[df.down_votes == 0]
df = df[df.up_votes > 0]
df = df.dropna( axis=0, how='any', thresh=None, subset=None, inplace=False)

In [3]:
females = df[df.gender == 'female'][:5002]

In [4]:
males = df[df.gender == 'male']

In [5]:
males['age'].value_counts()

twenties     9523
sixties      6632
thirties     6408
fourties     3926
teens        1975
fifties      1585
seventies     519
eighties      165
nineties       42
Name: age, dtype: int64

In [6]:
males['accent'].value_counts()

us             18533
england         4464
indian          2640
australia       2263
canada          1270
newzealand       514
african          300
singapore        229
philippines      185
ireland          183
other             90
malaysia          56
scotland          44
wales              2
hongkong           2
Name: accent, dtype: int64

In [7]:
teen = males[males.age == 'teens'][:833]

In [8]:
twe = males[males.age == 'twenties'][:833]
thir = males[males.age == 'thirties'][:833]
four = males[males.age == 'fourties'][:833]
fif = males[males.age == 'fifties'][:833]
six = males[males.age == 'sixties'][:833]

In [9]:
clean_men = teen
clean_men = clean_men.append([twe,thir,four,fif,six])

In [10]:
clean_men['accent'].value_counts()

us             2524
england        1014
australia       456
canada          377
indian          311
african         187
philippines      44
newzealand       33
scotland         29
ireland          22
other             1
Name: accent, dtype: int64

In [11]:
finalData = females
finalData = finalData.append(clean_men)

In [12]:
finalData['age'].value_counts()

thirties    2321
twenties    2081
sixties     1870
fifties     1504
fourties    1275
teens        949
Name: age, dtype: int64

In [13]:
finalData = finalData.drop(columns=['client_id', 'sentence','up_votes', 'down_votes'])
finalData.to_csv('whole10data.csv')

In [82]:
finalData.to_csv('whole10data.csv')

In [14]:
with open('clean10dataNoAccent.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([ "Image", "Tags"])
    for idx ,i in finalData.iterrows():
        writer.writerow([i['path'].replace('.mp3','.jpg'), "{0} {1}".format(i['age'],i['gender'])])
        

In [ ]:
def make_spectogram(audio_path):
    
    sig, fs = librosa.load(audio_path)   
    # make pictures name 
    save_path = 'sentiment-spectogram/' + audio_path[16:].replace('.wav','.jpg')

    pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    S = librosa.feature.melspectrogram(y=sig, sr=fs)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
    pylab.close()